# 🎮GameSelect


## ✅ Checklist
- [ ] Descrição do problema e público-alvo
- [ ] Carregamento do dataset `games_catalog_G4.csv`
- [ ] Dicionário de dados
- [ ] EDA (exploração, gráficos, estatísticas)
- [ ] Limpeza (ausentes/outliers) e justificativas
- [ ] Vetorização de gêneros (multi-hot)
- [ ] Similaridade cosseno
- [ ] Função de recomendação + casos de teste
- [ ] Insights e próximos passos


## 1. Descrição do Problema
* Objetivo: Recomendar jogos com base na similaridade do último titulo jogado
* Público alvo: Gamers ou pessoas interessadas em jogos
* Critério de sucesso: **recomendar 5 jogos similares** a um título dado

## 2. Carregando o Dataset e as Bibliotecas utilizadas

Carregando as blibiotecas necessárias

In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("https://raw.githubusercontent.com/fisicorj/aulacienciadedados/refs/heads/main/games_catalog_G4.csv")
df.tail()

,title,genres,platform,price,rating,reviews_count,release_year,age_rating
25,Brawl Stars,"Action,Multiplayer,MOBA",Mobile,0.00,4.51,45478,2020,12+
26,PUBG Mobile,"Shooter,Battle Royale,Multiplayer",Mobile,9.99,4.20,81055,2024,E
27,Pokemon GO,"AR,Casual,Adventure",Mobile,0.00,4.12,348312,2011,12+
28,Among Us,"Party,Multiplayer,Social Deduction",Mobile,0.00,4.38,151105,2018,12+
29,Call of Duty Mobile,"Shooter,Multiplayer,Action",Mobile,14.99,3.67,44281,2014,18+


## 4. EDA — Análise Exploratória

### 4.1 Estrutura e tipos

### 4.2 Estatísticas descritivas

### 4.3 Valores ausentes

### 4.4 Distribuições e contagens (exemplos)

### 4.5 Visualizações (use matplotlib)

## 5. Limpeza de Dados (ausentes e outliers)
Descreva o que foi feito e por quê.

## 6. Preparação para Recomendação — Vetorização de Gêneros

Exploda a coluna `genres` e crie uma matriz **multi-hot** (uma coluna por gênero).

## 7. Similaridade Cosseno

## 8. Função de Recomendação

## 9. Teste do Recomendador

## 10. Visualização dos Resultados

## 11. Insights e Próximos Passos